In [14]:
import pandas as pd

## read csv to dataframe
fb_df = pd.read_csv("fb.csv")
insta_df = pd.read_csv("insta.csv")
print(fb_df)
print(insta_df)

   FacebookTime        Day           Date  Year
0        6:57pm   Saturday     October 28  2020
1        1:12pm     Sunday    October 25   2020
2        4:10pm    Tuesday     October 13  2020
3        5:34pm  Wednesday   September 30  2020
4        6:34pm     Monday   September 28  2020
..          ...        ...            ...   ...
69       7:27pm   Saturday      August 03  2019
70       1:16am   Thursday        June 18  2020
71      12:03am    Tuesday        June 11  2019
72       4:51am    Tuesday        June 04  2019
73       4:51am    Tuesday        June 04  2019

[74 rows x 4 columns]
    InstagramTime        Day           Date  Year
0          7:30pm  Wednesday   September 30  2020
1          8:28pm   Saturday   September 26  2020
2          2:05pm   Saturday   September 12  2020
3          9:56pm     Friday   September 11  2020
4          6:44pm     Friday   September 11  2020
..            ...        ...            ...   ...
70         6:21pm   Saturday      August 03  2019
7

In [15]:
#1. Explore the frequency by days

fb_days = fb_df.groupby("Day")
insta_days = insta_df.groupby("Day")
days = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
day_df1 = pd.DataFrame()
day_df2 = pd.DataFrame()
# calculate frequency of number of times online for fb data
day_freq = []
for day in days:
    day_df1 = fb_days.get_group(day)["Day"]
    day_df2 = insta_days.get_group(day)["Day"]
    day_df1 = day_df1.append(day_df2,ignore_index=True)
    day_freq.append(len(day_df1))
    #print(day_freq)
print(day_freq)

[23, 25, 15, 18, 20, 24, 24]


In [16]:
## Calculate average number of times I am online on social media during weekend and weekday
sumweek = 0
for i in range(0,5):
    sumweek +=day_freq[i]
sumweekend = day_freq[5] + day_freq[6]

print("Weekdays frequency:",sumweek/5,"Weekend Frequency:",sumweekend/2)

Weekdays frequency: 20.2 Weekend Frequency: 24.0


### Hypothesis question: Am I more often on social media during the weekend than weekday?

Step 1:(Stating hypothesis)   
$H_o : \mu_{weekday} = \mu_{weekend}$  
$H_1 : \mu_{weekend} > \mu_{weekday}$  

Step 2:(Level of Significance)  
For testing purpose, I will just use a level of significance of 0.05

Step 3:(Test determined)  
This is a one tailed test two independent test  
df =  5 + 2-2 = 5  
t-value = 2.015  (If t computed is larger than 2.015, reject null hypothesis.Else do not.)

In [17]:
##Step 4:

import scipy.stats as stats
import numpy as np
import pandas as pd


alpha = 0.05
weekday_ser = pd.Series(day_freq[0:5])
print(weekday_ser)
weekend_ser = pd.Series(day_freq[5:7])
print(weekend_ser)


# ttest_ind performs a two-tailed test
t_computed, p_value = stats.ttest_ind(weekday_ser,weekend_ser)
print("t-computed:", t_computed, "p-value:", p_value/2)
if p_value/2 < alpha: 
    print("Reject H0, p-value:", p_value/2)
else:
    print("Fail to reject H0, p-value:", p_value/2)

0    23
1    25
2    15
3    18
4    20
dtype: int64
0    24
1    24
dtype: int64
t-computed: -1.2815623994965906 p-value: 0.128103804904969
Fail to reject H0, p-value: 0.128103804904969


Based on the results, I did not appear to be more active online during the weekday or weekends. t-computed is smaller than 2.105 and the p-value is larger than 0.05

In [18]:
## CLEAN DATA FOR TIME

# e.g: if 1:00am, 2:00am,3:00am,4:00am,5:00am or 6:00am will be hard to classify whether it is before or after 5:00pm
# so in this case, i think its best to disregard the data
# This anamolies exist because i suspect of the time difference between US and Malaysia. The data obtained contains the 
# day, month, year where I still have not come to the US. So, the time change may be why I am online during such odd hours

# clean facebook first
i =0 
for index, row in fb_df.iterrows():
    strtime = row['FacebookTime']
    if strtime.find("am")!= -1:
        if(int(strtime.split(':')[0]) < 7):
            fb_df.drop(index, inplace=True)
        elif(int(strtime.split(':')[0]) > 11):
            fb_df.drop(index, inplace=True)

for index, row in insta_df.iterrows():
    strtime = row[0]
    if strtime.find("am")!= -1:
        if(int(strtime.split(':')[0]) < 7):
            insta_df.drop(index, inplace=True)
        elif(int(strtime.split(':')[0]) > 11):
            insta_df.drop(index, inplace=True)

fb_df.to_csv("fbcleaned.csv",index=False)
insta_df.to_csv("instacleaned.csv",index=False)

In [19]:
## Group the time before and after 5:00pm for facebook
fb_am = 0
fb_pm = 0
fb_amser =[]
fb_pmser =[]
days_week = []

fb_days = fb_df.groupby('Day')
days = fb_days.groups.keys()

#group am and pm
for day in days:
    idf = fb_days.get_group(day)
    fb_am =0
    fb_pm =0
    for fbtime in idf.iloc[:,0]:
        if(fbtime.find("am")!= -1):
            fb_am +=1
        elif(fbtime.find("pm")!= -1): #it can be before 5:00pm like 1:00,2:00,3:00pm etc. so we have to compare
            if(fbtime[1] < "5"):
                fb_am +=1
            else:
                fb_pm +=1
    fb_amser.append(fb_am)
    fb_pmser.append(fb_pm)
    days_week.append(day)
            
print("Before 5:00pm:",fb_amser,"After 5:00pm:",fb_pmser)

## Group for time before and after 5:00pm for Instagram

insta_days = insta_df.groupby('Day')
days = insta_days.groups.keys()
print(days)
insta_am = 0
insta_pm = 0
insta_amser =[]
insta_pmser = []

#group am and pm
j= 0
for day in days:
    idf = insta_days.get_group(day)
    insta_am = 0
    insta_pm =0
    for instatime in idf.iloc[:,0]:
        if(instatime.find("am")!= -1):
            insta_am +=1
        elif(instatime.find("pm")!= -1): #it can be before 5:00pm like 1:00,2:00,3:00pm etc. so we have to compare
            if(instatime[1] < "5"):
                insta_am+=1
            else:
                insta_pm +=1
                
    insta_amser.append(insta_am)
    insta_pmser.append(insta_pm)
        

print("Before 5:00pm:",insta_amser,"After 5:00pm:",insta_pmser)


Before 5:00pm: [5, 7, 3, 10, 6, 5, 1] After 5:00pm: [3, 6, 9, 4, 2, 2, 4]
dict_keys(['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday'])
Before 5:00pm: [5, 3, 8, 6, 6, 7, 5] After 5:00pm: [6, 4, 4, 2, 2, 5, 5]


In [20]:
## Total for both social media before 5:00pm and after 5:00pm 

before_5 = np.array(fb_amser) + np.array(insta_amser)
after_5 = np.array(fb_pmser) + np.array(insta_pmser)
print("Before 5:",before_5,"After 5:",after_5)


Before 5: [10 10 11 16 12 12  6] After 5: [ 9 10 13  6  4  7  9]


### Hypothesis question: Am I more active online on Social media during the day(before 5:00pm) than night(after 5:00pm)

Step 1:(Stating hypothesis and null hypothesis)  
$H_o: \mu_{before5} = \mu_{after5}$  
$H_o : \mu_{before5} > \mu_{after5}$  

Step 2:(Choosing significance level)  
I will choose a significance level of 0.10

Step 3:(Identifying type of test to use)  
We will use a two sample independent test but one tailed because a direction is present  
df = 7 + 7 -2  
t- value = 1.385
If t-computed is larger than t-value, we reject the null hypothesis; else do not.

In [21]:
## Step 4:
alpha = 0.10
# ttest_ind performs a two-tailed test
t_computed, p_value = stats.ttest_ind(before_5,after_5)
print("t-computed:", t_computed, "p-value:", p_value/2)
if p_value/2 < alpha: 
    print("Reject H0, p-value:", p_value/2)
else:
    print("Fail to reject H0, p-value:", p_value/2)

t-computed: 1.7131723058681207 p-value: 0.05618635306216984
Reject H0, p-value: 0.05618635306216984


Since the p value is smaller than 0.10, and the t-computed is larger than 1.385, we reject the null hypothesis. Therefore, I am more active on social media during the day than night